In [4]:
import sagemaker

print(sagemaker.__version__)
role = sagemaker.get_execution_role()

2.101.1


In [5]:
bucket = "sagemaker-transformer"
prefix = "Train"
classes = "['Farm','Wild']"

In [10]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.23-1',
                                     role=role,
                                     instance_type='ml.m5.4xlarge',
                                     instance_count=1)

In [ ]:
%%time
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='image-processing.py',
                      outputs=[
                          ProcessingOutput(
                              output_name='train_data',
                              source='/opt/ml/processing/train'),
                          ProcessingOutput(
                              output_name='valid_data',
                              source='/opt/ml/processing/valid'),
                          ProcessingOutput(
                              output_name='test_data',
                              source='/opt/ml/processing/test')
                      ],
                      arguments=[
                          '--bucket', bucket,
                          '--prefix', prefix,
                          '--classes', classes
                      ]
                     )

In [15]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://sagemaker-eu-central-1-843182712965/sagemaker-scikit-learn-2022-09-03-18-17-21-834/output/train_data
s3://sagemaker-eu-central-1-843182712965/sagemaker-scikit-learn-2022-09-03-18-17-21-834/output/valid_data
s3://sagemaker-eu-central-1-843182712965/sagemaker-scikit-learn-2022-09-03-18-17-21-834/output/test_data
